In [9]:
#Import libraries
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import PIL as pil
import os
import PIL
import pathlib
import glob

#Set to GPU
devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(devices[0], True)

#Path
images_source = pathlib.Path("D:\s4532664\COMP3710\ISIC2018_Task1-2_Training_Data")

#Input images
training_dir = images_source / "ISIC2018_Task1-2_Training_Input_x2\*.jpg"

#Segmentation images (label images)
seg_training_dir = images_source / "ISIC2018_Task1_Training_GroundTruth_x2\*.png"
                             


In [10]:
print(training_dir)
print(seg_training_dir)


D:\s4532664\COMP3710\ISIC2018_Task1-2_Training_Data\ISIC2018_Task1-2_Training_Input_x2\*.jpg
D:\s4532664\COMP3710\ISIC2018_Task1-2_Training_Data\ISIC2018_Task1_Training_GroundTruth_x2\*.png


In [11]:
def load_data():
    
    input_files = sorted(glob.glob(str(training_dir)))
    seg_files = sorted(glob.glob(str(seg_training_dir)))

    print("Input images:", len(input_files), "Segmentation images:", len(seg_files))



In [12]:
load_data()

Input images: 2594 Segmentation images: 2594
